In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
print ("pandas version:  {}".format(pd.__version__))
import time
from sklearn.preprocessing import LabelEncoder
import matplotlib.pyplot as plt

import sklearn
from sklearn import linear_model
print("sklearn version:  {}".format(sklearn.__version__))
from sklearn.metrics import mean_squared_log_error

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

pandas version:  2.0.3
sklearn version:  1.2.2
/kaggle/input/storesalestfs-featureenigneering/__results__.html
/kaggle/input/storesalestfs-featureenigneering/__notebook__.ipynb
/kaggle/input/storesalestfs-featureenigneering/__output__.json
/kaggle/input/storesalestfs-featureenigneering/train_features.csv
/kaggle/input/storesalestfs-featureenigneering/test_features.csv
/kaggle/input/storesalestfs-featureenigneering/custom.css
/kaggle/input/store-sales-time-series-forecasting/oil.csv
/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv
/kaggle/input/store-sales-time-series-forecasting/holidays_events.csv
/kaggle/input/store-sales-time-series-forecasting/stores.csv
/kaggle/input/store-sales-time-series-forecasting/train.csv
/kaggle/input/store-sales-time-series-forecasting/test.csv
/kaggle/input/store-sales-time-series-forecasting/transactions.csv


Most of the structure is copied from the LGBM notebook for this competition.

In [2]:
# read the output file of the prepare data notebook here
train = pd.read_csv('/kaggle/input/storesalestfs-featureenigneering/train_features.csv')
test = pd.read_csv('/kaggle/input/storesalestfs-featureenigneering/test_features.csv')
# and sample submission
sample_submission = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/sample_submission.csv')

In [3]:
train.head()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,dayofweek,...,c_sales_std,is_onpromotion,op_sales_median,op_sales_mean,op_sales_min,op_sales_max,dcoilwtico,is_national_hol,is_local_hol,is_holiday
0,648648,2014-01-01,1,AUTOMOTIVE,0.0,0,2014,1,1,2,...,6.250549,0,3.0,3.542113,0.0,19.0,98.17,True,False,True
1,648649,2014-01-01,1,BABY CARE,0.0,0,2014,1,1,2,...,0.766963,0,0.0,0.000000,0.0,0.0,98.17,True,False,True
2,648650,2014-01-01,1,BEAUTY,0.0,0,2014,1,1,2,...,5.110346,0,2.0,2.263397,0.0,11.0,98.17,True,False,True
3,648651,2014-01-01,1,BEVERAGES,0.0,0,2014,1,1,2,...,2431.441640,0,1093.0,1266.943522,0.0,3366.0,98.17,True,False,True
4,648652,2014-01-01,1,BOOKS,0.0,0,2014,1,1,2,...,0.618165,0,0.0,0.159848,0.0,8.0,98.17,True,False,True


In [4]:
test.head()

,id,date,store_nbr,family,sales,onpromotion,year,month,day,dayofweek,...,c_sales_std,is_onpromotion,op_sales_median,op_sales_mean,op_sales_min,op_sales_max,dcoilwtico,is_national_hol,is_local_hol,is_holiday
0,3000888,2017-08-16,1,AUTOMOTIVE,0.0,0,2017,8,16,2,...,6.250549,0,3.0,3.542113,0.0,19.0,46.8,False,False,False
1,3000889,2017-08-16,1,BABY CARE,0.0,0,2017,8,16,2,...,0.766963,0,0.0,0.000000,0.0,0.0,46.8,False,False,False
2,3000890,2017-08-16,1,BEAUTY,0.0,2,2017,8,16,2,...,5.110346,1,4.0,4.173516,1.0,12.0,46.8,False,False,False
3,3000891,2017-08-16,1,BEVERAGES,0.0,20,2017,8,16,2,...,2431.441640,1,2097.0,1942.677134,325.0,5051.0,46.8,False,False,False
4,3000892,2017-08-16,1,BOOKS,0.0,0,2017,8,16,2,...,0.618165,0,0.0,0.159848,0.0,8.0,46.8,False,False,False


In [5]:
# append train and test temporarily
train = pd.concat( [train,test])

In [6]:
# convert text columns to numbers
label_encoder = LabelEncoder()

train.city = label_encoder.fit_transform(train.city)
train.state = label_encoder.fit_transform(train.state)
train.cluster = label_encoder.fit_transform(train.cluster)
train.type = label_encoder.fit_transform(train.type)
train.family = label_encoder.fit_transform(train.family)
train.is_national_hol = label_encoder.fit_transform(train.is_national_hol)
train.is_local_hol = label_encoder.fit_transform(train.is_local_hol)
train.is_holiday = label_encoder.fit_transform(train.is_holiday)
train.is_weekend = label_encoder.fit_transform(train.is_weekend)
train.dtypes

id                     int64
date                  object
store_nbr              int64
family                 int64
sales                float64
onpromotion            int64
year                   int64
month                  int64
day                    int64
dayofweek              int64
dayofyear              int64
weekofyear             int64
is_weekend             int64
weekIdContinous        int64
sales_lag2w          float64
sales_lag4w          float64
sales_lag8w          float64
sales_lag1y          float64
is_payday              int64
sf_monthly_mean      float64
sf_yearly_mean       float64
s_sales_median       float64
s_sales_mean         float64
s_sales_max          float64
s_sales_std          float64
city                   int64
state                  int64
type                   int64
cluster                int64
transactions_mean    float64
transactions_std     float64
st_ratio             float64
c_sales_median       float64
c_sales_mean         float64
c_sales_max   

In [7]:
# convert whole numbers to smaller data type 
train.store_nbr = train.store_nbr.astype('int8')
train.month = train.month.astype('int8')
train.day = train.day.astype('int8')
train.weekofyear = train.weekofyear.astype('int8')
train.dayofyear = train.dayofyear.astype('int16')
train.year = train.year.astype('int16')
train.family = train.family.astype('int8')
train.onpromotion = train.onpromotion.astype('int16')
train.is_onpromotion = train.is_onpromotion.astype('int8')
train.dayofweek = train.dayofweek.astype('int8')
train.city = train.city.astype('int8')
train.state = train.state.astype('int8')
train.type = train.type.astype('int8')
train.cluster = train.cluster.astype('int8')
train.is_national_hol = train.is_national_hol.astype('int8')
train.is_local_hol = train.is_local_hol.astype('int8')
train.is_holiday = train.is_holiday.astype('int8')
train.is_weekend = train.is_weekend.astype('int8')
train.is_payday = train.is_payday.astype('int8')
train.dcoilwtico = train.dcoilwtico.astype('float32')

In [8]:
train.isna().sum()

id                   0
date                 0
store_nbr            0
family               0
sales                0
onpromotion          0
year                 0
month                0
day                  0
dayofweek            0
dayofyear            0
weekofyear           0
is_weekend           0
weekIdContinous      0
sales_lag2w          0
sales_lag4w          0
sales_lag8w          0
sales_lag1y          0
is_payday            0
sf_monthly_mean      0
sf_yearly_mean       0
s_sales_median       0
s_sales_mean         0
s_sales_max          0
s_sales_std          0
city                 0
state                0
type                 0
cluster              0
transactions_mean    0
transactions_std     0
st_ratio             0
c_sales_median       0
c_sales_mean         0
c_sales_max          0
c_sales_std          0
is_onpromotion       0
op_sales_median      0
op_sales_mean        0
op_sales_min         0
op_sales_max         0
dcoilwtico           0
is_national_hol      0
is_local_ho

In [9]:
# split train and test again
test = train.loc[train.id >= 3000888]
test.shape

(28512, 45)

In [10]:
train = train.loc[train.id < 3000888]
train.shape

(2361414, 45)

# Model: Ridge Regression (regularized Regression)

In [11]:
columns_to_drop = ['sales','id','date','is_national_hol','is_local_hol']

In [12]:
# create validation set based on time, I use 2 weeks because the test set is 2 weeks
#train.date.max()
X_train = train.loc[train.date < '2017-08-01']
X_train.tail()
y_train = X_train.sales
X_train = X_train.drop(columns_to_drop, axis = 1)
X_val = train.loc[train.date >= '2017-08-01']
y_val = X_val.sales
X_val = X_val.drop(columns_to_drop, axis = 1)

In [13]:
X_val.head()

,store_nbr,family,onpromotion,year,month,day,dayofweek,dayofyear,weekofyear,is_weekend,...,c_sales_mean,c_sales_max,c_sales_std,is_onpromotion,op_sales_median,op_sales_mean,op_sales_min,op_sales_max,dcoilwtico,is_holiday
2334684,1,0,0,2017,8,1,1,213,31,0,...,6.491400,255.0,6.250549,0,3.0,3.542113,0.0,19.0,49.189999,0
2334685,1,1,0,2017,8,1,1,213,31,0,...,0.141260,116.0,0.766963,0,0.0,0.000000,0.0,0.0,49.189999,0
2334686,1,2,0,2017,8,1,1,213,31,0,...,4.076740,136.0,5.110346,0,2.0,2.263397,0.0,11.0,49.189999,0
2334687,1,3,26,2017,8,1,1,213,31,0,...,2738.754588,25413.0,2431.441640,1,2097.0,1942.677134,325.0,5051.0,49.189999,0
2334688,1,4,0,2017,8,1,1,213,31,0,...,0.090614,51.0,0.618165,0,0.0,0.159848,0.0,8.0,49.189999,0


In [14]:
regr = linear_model.Ridge(alpha= 100 )
regr.fit(X_train, y_train)

Ridge(alpha=100)

I just tested a few alphas manually....

In [15]:
preds = regr.predict(X_val)

In [16]:
preds.min()

-29.67895323166158

In [17]:
preds = preds.clip(0,15000)

In [18]:
rmsle_val = mean_squared_log_error(preds,y_val,squared=False)
rmsle_val

1.3111349314762322

In [19]:
# retrain on whole data
regr.fit(train.drop(columns = columns_to_drop),train.sales)

Ridge(alpha=100)

In [20]:
preds = regr.predict(test.drop(columns = columns_to_drop))

In [21]:
preds.min()

-63.85977796759107

In [22]:
preds = preds.clip(0,15000)

In [23]:
regr.coef_

array([ 2.25993366e-02, -1.40625310e-02,  3.07642930e-02,  6.71039744e+01,
        1.38436130e+02,  2.63106558e+00, -1.09347661e+00, -3.98802093e+00,
       -2.82318510e+00,  6.98622811e+01, -1.31865995e+00,  2.09969335e-01,
        1.48374998e-01,  1.70313194e-01,  5.53799244e-02,  4.89580673e+00,
        7.29438308e-01, -3.62520537e-01,  4.46604920e-02, -5.71757100e-02,
       -3.01044580e-05,  2.17016527e-02,  2.32090970e-02, -9.97316386e-03,
        2.56806985e-01,  2.19111582e-02,  8.31676736e-04,  1.89379586e-03,
        1.28205749e+00,  2.48939552e-02, -4.98132743e-02, -1.96404594e-04,
        4.25362734e-02,  5.84704231e+00,  4.27436029e-02,  2.40951133e-02,
       -5.38703790e-03,  3.69828228e-04,  1.67442141e-01,  3.26683477e+01])

In [24]:
features = train.drop(columns = columns_to_drop).columns

In [25]:
feature_coeff = pd.DataFrame(data={'features':features,'coef':regr.coef_})
#feature_coeff.plot(kind='bar')
#plt.show()
feature_coeff

,features,coef
0,store_nbr,0.022599
1,family,-0.014063
2,onpromotion,0.030764
3,year,67.103974
4,month,138.436130
5,day,2.631066
6,dayofweek,-1.093477
7,dayofyear,-3.988021
8,weekofyear,-2.823185
9,is_weekend,69.862281


In [26]:
regr.intercept_

-135223.63264949838

# Postprocessing

In [27]:
test = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/test.csv')
train = pd.read_csv('/kaggle/input/store-sales-time-series-forecasting/train.csv')

submission_pre = pd.DataFrame(data={"id" : sample_submission.id,
                                   "preds" : preds})

submission_pre = pd.merge(test,submission_pre, on='id',how='left')

# get info about which stores have 0 sales for which categories out of train
store_info=train[['store_nbr','family','sales']].groupby(['store_nbr','family']).sum().reset_index()
store_info = store_info.loc[store_info.sales == 0] #reduce to those without sales

categories_not_sold_in_all_stores = store_info.family.unique()

# set all predictions to 0 for the categories that are not sold in certain stores
for cat in categories_not_sold_in_all_stores:
    category = cat
    stores_that_dont_sell_category = list(store_info.loc[(store_info.family == category)].store_nbr)
    submission_pre.loc[submission_pre.store_nbr.isin(stores_that_dont_sell_category) & (submission_pre.family==category),['preds']] = 0
    
submission_pre.head()

,id,date,store_nbr,family,onpromotion,preds
0,3000888,2017-08-16,1,AUTOMOTIVE,0,0.000000
1,3000889,2017-08-16,1,BABY CARE,0,0.000000
2,3000890,2017-08-16,1,BEAUTY,2,0.000000
3,3000891,2017-08-16,1,BEVERAGES,20,2240.337004
4,3000892,2017-08-16,1,BOOKS,0,0.000000


# Submission

In [28]:
# generate submission file 
submission = submission_pre[['id','preds']]
submission = submission.rename(columns={'preds':'sales'})

submission.to_csv('submission.csv', index=False)
submission.head()

,id,sales
0,3000888,0.000000
1,3000889,0.000000
2,3000890,0.000000
3,3000891,2240.337004
4,3000892,0.000000
